# Reacher Continuous Control

## The Problem
We are training reinforcement learning agents with double-jointed arms to follow moving targets in the Unity ML-Agents Reacher environment. A reward of +0.1 is given for each time step that an agent's hand is in the target position. We want to maximize the number of time steps that the agent maintains its position in the target position.

### The Environment
We are using the Reacher environment from the Unity ML-Agents plugin.

For this task, each state consists of 33 variables corresponding to position, rotation, velocity, and angular velocities of the arm. Each action is a vector with four numbers in the range \[-1, 1\], corresponding to torque applicable to two joints. We simultaneously train 20 identical agents, each with their own copy of the environment. At each episode, we store the average of the scores accumulated by all the agents in a buffer of size 100. The task is considered solved when the average score of the buffer reaches or exceeds 30.

### Set Up
Make sure [Unity Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) is installed. Then run the following line to install all other required packages.

In [ ]:
!pip install -r requirements.txt

Next, import all necessary libraries.

In [ ]:
from collections import deque, namedtuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from unityagents import UnityEnvironment

To load the Unity environment, change the ```file_name``` parameter based on your operating system, as specified below:
 * Windows (x86): "envs/Reacher_Windows_x86/Reacher.exe"
 * Windows (x86_64): "envs/Reacher_Windows_x86_64/Reacher.exe"
 * Linux (x86): "envs/Reacher_Linux/Reacher.x86"
 * Linux (x86_64): "envs/Reacher_Linux/Reacher.x86_64"
 * Mac: "envs/Reacher.app"

For example,
```
env = UnityEnvironment(file_name="envs/Reacher_Linux/Reacher.x86_64")
```

In [ ]:
env = UnityEnvironment(file_name="envs/Reacher_Linux/Reacher.x86_64")

To examine the environment in more detail, run the following code chunk. As can be seen, there are 20 agents, each state consists of 37 components, and each action consists of 4 components.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

We are now ready to build the agent.

## The Solution

### Deep Deterministic Policy Gradient (DDPG)
We implement the architecture presented in [this paper (Lillicrap et al, 2015)](https://arxiv.org/pdf/1509.02971.pdf) on Deep Deterministic Policy Gradients, which uses an Actor-Critic dual-network structure to combine both policy gradient methods (the actor) and action-value approximations (the critic) into a model-free framework that achieves satisfactory performance in continuous action spaces. The structure of both networks is shown below.

In [ ]:
def fan_in(size):  # helper method for initializing weights
    f = size[0]
    bound = 1.0 / np.sqrt(f)
    return torch.Tensor(size).uniform_(-bound, bound)

class Actor(nn.Module):
    def __init__(self, state_size, action_size, hidden1=256, hidden2=128, final_weights_init=3e-3):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.final_weights_init = final_weights_init
        self.fc1 = nn.Linear(self.state_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, self.action_size)
        self.init_weights()
        
    def init_weights(self):
        self.fc1.weight.data = fan_in(self.fc1.weight.data.size())
        self.fc2.weight.data = fan_in(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-self.final_weights_init, self.final_weights_init)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))

class Critic(nn.Module):
    def __init__(self, state_size, action_size, hidden1=256, hidden2=128, final_weights_init=3e-3):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.final_weights_init = final_weights_init
        self.fc1 = nn.Linear(self.state_size, hidden1)
        self.fc2 = nn.Linear(hidden1+self.action_size, hidden2)
        self.fc3 = nn.Linear(hidden2, self.action_size)
        self.init_weights()
        
    def init_weights(self):
        self.fc1.weight.data = fan_in(self.fc1.weight.data.size())
        self.fc2.weight.data = fan_in(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-self.final_weights_init, self.final_weights_init)
        
    def forward(self, state_action):
        state, action = state_action
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(torch.cat([x, action], 1)))
        return self.fc3(x)

### Experience Replay
Like Deep Q-Networks (DQN), the DDPG agent also makes use of experience replay, where interactions with the environment are stored in _(state, action, reward, next_state, done)_ tuples in a buffer of capacity 1e5 and sampled at random in decorrelated minibatches of size 128 to use in updating all the networks. The structure of the replay buffer is shown below.

In [ ]:
class ReplayBuffer():
    def __init__(self, buffer_size, minibatch_size, device):
        self.buffer = deque(maxlen=buffer_size)
        self.minibatch_size = minibatch_size
        self.device = device
        self.experience = namedtuple("Experience", field_names=['state', 'action', 'reward', 'next_state', 'done'])
        self.size = 0
        
    def add(self, state, action, reward, next_state, done):
        self.buffer.append(self.experience(state, action, reward, next_state, done))
        self.size += 1
        
    def sample(self):
        experiences = random.sample(self.buffer, k=self.minibatch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([1 if e.done else 0 for e in experiences if e is not None])).float().to(self.device)
        return (states, actions, rewards, next_states, dones)
        
    def get_len(self):
        return self.size

### Random Noise for Action Exploration

The DDPG paper makes use of the Ornstein-Uhlenbeck Process for generating random noise for momentum-based processes. This noise will encourage the agent to explore new actions, and the magnitude of the noise decreases over time as the variance decays with each episode, down to a minimum value of 0.05, where it remains for the remainder of training. For this task, we use parameters of mu=0, theta=0.15, sigma=0.2, and sigma_decay=0.98.

In [ ]:
class OrnsteinUhlenbeck():
    def __init__(self, num_agents, action_size, mu=0, theta=0.15, sigma=0.2, sigma_min=0.05, sigma_decay=0.98):
        self.num_agents = num_agents
        self.action_size = action_size
        self.mu = mu
        self.sigma = sigma
        self.sigma_min = sigma_min
        self.sigma_decay = sigma_decay
        self.theta = theta
        self.prev_val = np.zeros((self.num_agents, self.action_size))
    
    def sample(self):
        val = self.prev_val + self.theta * (self.mu - self.prev_val) * self.sigma * np.random.normal(size=(self.num_agents, self.action_size))
        self.prev_val = val
        return val
    
    def reset(self):
        self.prev_val = np.zeros((self.num_agents, self.action_size))
        self.sigma = max(self.sigma_min, self.sigma*self.sigma_decay)

### The Agent

The RL agent is shown below. It has two copies of each of the actor and the critic networks; while all the target and local networks are updated at every time step, the target networks are updated much more slowly using the *soft\_update* function, parametrized by tau=0.001. This reduces unwanted sequential dependencies in the data that could cause the networks to update their parameters based on moving targets, and contributes to more stability during training. The actor is updated with a policy loss, and the critic is updated using a Mean-Squared Error loss for the TD error (with gamma=0.99). Although the code allows for updating the networks every *update\_freq* steps, using *update\_freq*=1 appears to have the best results.

We use Adam optimizers with learning rates of 1e-4 to train both the actor and the critic networks.

In [ ]:
class DDPG_Agent():
    def __init__(self, num_agents, state_size, action_size, buffer_capacity=1e5, minibatch_size=128, update_freq=1, tau=1e-3, gamma=0.99, lr_actor=1e-4, lr_critic=1e-4):
        self.num_agents = num_agents
        self.state_size = state_size
        self.action_size = action_size
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.buffer = ReplayBuffer(int(buffer_capacity), minibatch_size, self.device)
        self.noise = OrnsteinUhlenbeck(num_agents=num_agents, action_size=action_size)
        self.num_steps = 0
        
        self.actor_local = Actor(state_size, action_size).to(self.device)
        self.actor_target = Actor(state_size, action_size).to(self.device)
        self.copy_weights(self.actor_target, self.actor_local)
        self.optim_actor = optim.Adam(self.actor_local.parameters(), lr=lr_actor)
        self.critic_local = Critic(state_size, action_size).to(self.device)
        self.critic_target = Critic(state_size, action_size).to(self.device)
        self.copy_weights(self.critic_target, self.critic_local)
        self.optim_critic = optim.Adam(self.critic_local.parameters(), lr=lr_critic) #, weight_decay=1e-2)
        
        self.update_freq = update_freq
        self.minibatch_size = minibatch_size
        self.tau = tau
        self.gamma = gamma
        
    def copy_weights(self, target_network, source_network):
        for target_param, param_to_copy in zip(target_network.parameters(), source_network.parameters()):
            target_param.data.copy_(param_to_copy.data)
    
    def select_action(self, states):  # states is shape (num_agents, state_size)
        states = torch.from_numpy(states).float().to(self.device)
        self.actor_local.eval()
        with torch.no_grad():
            actions = self.actor_local(states)  # shape (num_agents, action_size)
        self.actor_local.train()
        return np.clip(actions.cpu().data.numpy() + self.noise.sample(), -1, 1)
    
    def update(self, states, actions, rewards, next_states, dones):
        for i in range(self.num_agents):
            self.buffer.add(states[i], actions[i], rewards[i], next_states[i], dones[i])
        self.num_steps = (self.num_steps + 1) % self.update_freq
        if self.num_steps == 0 and self.buffer.get_len() >= self.minibatch_size:
            self.update_local()
            self.soft_update(self.actor_target, self.actor_local)
            self.soft_update(self.critic_target, self.critic_local)
    
    def update_local(self):
        states, actions, rewards, next_states, dones = self.buffer.sample()
        
        # update critic
        next_actions_target = self.actor_target(next_states)
        future_rewards_target = self.critic_target((next_states, next_actions_target))
        value_target = rewards + self.gamma*future_rewards_target*(1-dones)
        value_local = self.critic_local((states, actions))
        critic_loss = F.mse_loss(value_local, value_target)
        self.optim_critic.zero_grad()
        critic_loss.backward()
        self.optim_critic.step()
        
        # update actor
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local((states, actions_pred)).mean()
        self.optim_actor.zero_grad()
        actor_loss.backward()
        self.optim_actor.step()
    
    def soft_update(self, target_network, local_network):
        for target_param, local_param in zip(target_network.parameters(), local_network.parameters()):
            target_param.data.copy_(self.tau*local_param.data + (1-self.tau)*target_param.data)

### Putting it Together

Now we are ready to train the agents. We use a cap of 500 episodes, and allow each episode to play out from start to finish. At the end of each episode, the average score from the 20 agents in that episode is stored into the buffer *score\_window*, and the task is completed when the average of the values within *score\_window* reaches or exceeds +30. The actor and critic models are then saved into _actor.pth_ and _critic.pth_, respectively, and the learning curve is plotted to show the average score across the agents at every training episode. This figure is saved into *learning\_curve.png*.

In [ ]:
# reset environment
env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations
num_agents, state_size = states.shape
action_size = brain.vector_action_space_size

num_episodes = 500

agent = DDPG_Agent(num_agents=num_agents, state_size=state_size, action_size=action_size)
score_window = deque(maxlen=100)
score_record = []
highest_avg_score = 0

print(f'Training starts with {num_agents} agents!')
for ep in range(1, num_episodes+1):
    scores = np.zeros(num_agents)
    agent.noise.reset()
    env_info = env.reset(train_mode=True)[brain_name]
    while True:
        actions = agent.select_action(states)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        scores += rewards
        dones = env_info.local_done
        agent.update(states, actions, rewards, next_states, dones)
        states = next_states
        if np.any(dones):
            break
    avg_score = np.mean(scores)
    score_window.append(avg_score)
    score_record.append(avg_score)
    if avg_score > highest_avg_score:
        highest_avg_score = avg_score
    print(f'\rEpisode {ep}/{num_episodes}: Average Score = {avg_score}, Highest Average Score = {highest_avg_score}', end="")
    if np.mean(score_window) >= 30:
        print(f'\nEnvironment solved in {ep} episodes!   Average Score: {np.mean(score_window)}')
        break
        
# save the trained model
torch.save(agent.actor_local.state_dict(), 'actor.pth')
torch.save(agent.critic_local.state_dict(), 'critic.pth')

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(score_record)), score_record)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('learning_curve.png')

One of the runs solved this task in 164 episodes, and the models for that run are saved in _actor.pth_ and _critic.pth_. The learning curve is pictured below:

![learning curve](learning_curve.png)

As can be seen, the curve rises steadily upward, and the agents are able to achieve an average score of +30 in a single episode by around episode 100. The scores continue to grow until about episode 110, where the average score per episode stabilizes to between about 33 to 38. The task is solved by episode 164.

After the task is solved, we can close the environment.

In [ ]:
env.close()

Future work involves more fine-grained hyperparameter tuning and a comparison of the performance of other reinforcement learning methods like A3C, A2C, PPO, and D4PG. We can also explore the generalization capacity of this DDPG architecture with its current set of hyperparameters, and test it on a variety of other tasks.